<a href="https://colab.research.google.com/github/IanQS/neuromatch_project/blob/main/steinmetz_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling of the Steinmetz dataset

- uses [Neuromatch Load Steinmetz Decisions](https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/main/projects/neurons/load_steinmetz_decisions.ipynb#scrollTo=DJ-jzsE5eLxX) as a base

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.decomposition import PCA
import concurrent.futures
from multiprocessing import Pool

!pip install -q ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
time: 387 µs (started: 2023-07-19 23:30:30 +00:00)


In [2]:
# @title Data Downloading And Stacking
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

all_ds = np.array([])
for j in range(len(fname)):
  all_ds = np.hstack((all_ds,
                      np.load('steinmetz_part%d.npz'%j,
                              allow_pickle=True)['dat']))

time: 51.5 s (started: 2023-07-19 23:30:30 +00:00)


# Dataset Description

(taken and modified from the Neuromatch Load Steinmetz Decisions notebook)

## High-level

`all_ds` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `curr_ds = all_ds[k]`, you have the fields below. For extra variables, check out the extra notebook and extra data files (lfp, waveforms and exact spike times, non-binned).

## Fields present (not all are used)

* `curr_ds['mouse_name']`: mouse name
* `curr_ds['date_exp']`: when a session was performed
* `curr_ds['spks']`: neurons by trials by time bins.    
* `curr_ds['brain_area']`: brain area for each neuron recorded.
* `curr_ds['ccf']`: Allen Institute brain atlas coordinates for each neuron.
* `curr_ds['ccf_axes']`: axes names for the Allen CCF.
* `curr_ds['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `curr_ds['contrast_left']`: contrast level for left stimulus.
* `curr_ds['gocue']`: when the go cue sound was played.
* `curr_ds['response_time']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `curr_ds['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response.
* `curr_ds['feedback_time']`: when feedback was provided.
* `curr_ds['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `curr_ds['wheel']`: turning speed of the wheel that the mice uses to make a response, sampled at `10ms`.
* `curr_ds['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position.
* `curr_ds['face']`: average face motion energy from a video camera.
* `curr_ds['licks']`: lick detections, 0 or 1.   
* `curr_ds['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons".
* `curr_ds['%X%_passive']`: same as above for `X` = {`spks`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses.
* `curr_ds['prev_reward']`: time of the feedback (reward/white noise) on the previous trial in relation to the current stimulus time.
* `curr_ds['reaction_time']`: ntrials by 2. First column: reaction time computed from the wheel movement as the first sample above `5` ticks/10ms bin. Second column: direction of the wheel movement (`0` = no move detected).  


The original dataset is here: https://figshare.com/articles/dataset/Dataset_from_Steinmetz_et_al_2019/9598406

In [3]:
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
region_colors = ['blue', 'red', 'green', 'darkblue', 'violet', 'lightblue', 'orange', 'gray']
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"],  # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"],  # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP","TT"],  # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"],  # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"],  # basal ganglia
                ["BLA", "BMA", "EP", "EPd", "MEA"]  # cortical subplate
                ]

# Assign each area an index
area_to_index = dict(root=0)
counter = 1
for group in brain_groups:
    for area in group:
        area_to_index[area] = counter
        counter += 1

# Figure out which areas are in each dataset
areas_by_dataset = np.zeros((counter, len(all_ds)), dtype=bool)
for j, d in enumerate(all_ds):
    for area in np.unique(d['brain_area']):
        i = area_to_index[area]
        areas_by_dataset[i, j] = True


time: 7.95 ms (started: 2023-07-19 23:31:22 +00:00)


In [25]:
DATASET_IDX = 11
curr_ds = all_ds[DATASET_IDX]

dt = curr_ds["bin_size"]
NUM_NEURONS_RECORDED = curr_ds["spks"].shape[0]
NUM_TRIALS = curr_ds["spks"].shape[1]
NUM_BINNED_TIMES = curr_ds["spks"].shape[2]

if DATASET_IDX != 11:
    raise Exception("Code is only meant for DATASET_IDX=11")
else:
    NUM_REGIONS = 4
    NUM_NEURONS_RECORDED = len(curr_ds["brain_area"])  # The string idx version of

brain_subregions = NUM_REGIONS * np.ones(NUM_NEURONS_RECORDED, )  # last one is "other"
for j in range(NUM_REGIONS):
  brain_subregions[
      np.isin(curr_ds['brain_area'], brain_groups[j])
      ] = j  # assign a number to each region


time: 2.47 ms (started: 2023-07-19 23:46:41 +00:00)


# Creating the dataset

In [5]:
LABELS = curr_ds["response"]  # RIGHT - NO_GO - LEFT (-1, 0, 1)
y = LABELS

time: 568 µs (started: 2023-07-19 23:31:22 +00:00)


In [6]:
def log_shapes():
    _ds = curr_ds['spks']
    print(f"All spikes shape: {_ds.shape}")
    _ds_brain_region = _ds[brain_subregions == 0]
    print(f"\t- Spike shape for sample brain region (0-th): {_ds_brain_region.shape}")

    _ds_0th_left_response = _ds_brain_region[:, y >= 0]
    print(f"\t- Spike shape for sample brain region (0-th) left responses: {_ds_0th_left_response.shape}")

    averaged_over_left_response = _ds_0th_left_response.mean(axis=(0, 1))
    print(f"\t- Averaged brain region (0-th) left responses: {averaged_over_left_response.shape}")

log_shapes()


All spikes shape: (698, 340, 250)
	- Spike shape for sample brain region (0-th): (145, 340, 250)
	- Spike shape for sample brain region (0-th) left responses: (145, 199, 250)
	- Averaged brain region (0-th) left responses: (250,)
time: 25.6 ms (started: 2023-07-19 23:31:22 +00:00)


In [7]:
def create_design_data():
    spike_partitioned = {}  # brain region to spike mapping
    for i in range(NUM_REGIONS):
        spks = curr_ds["spks"][brain_subregions == j]
        spikes_for_left_response = spks[:, y < 0]
        spikes_for_right_response = spks[:, y > 0]
        spikes_for_no_response = spks[:, y == 0]

        spike_partitioned[regions[i]] = [
            spikes_for_left_response,
            spikes_for_no_response,
            spikes_for_right_response
        ]
    return spike_partitioned

_Xs = create_design_data()

time: 24.6 ms (started: 2023-07-19 23:31:22 +00:00)


In [8]:
print(f"Left-Resp Visual Cortex shape: {Xs['vis ctx'][0].shape}")
print(f"No-Resp Visual Cortex shape: {Xs['vis ctx'][1].shape}")
print(f"Right-Resp Visual Cortex shape: {Xs['vis ctx'][2].shape}")

Left-Resp Visual Cortex shape: (78, 141, 250)
No-Resp Visual Cortex shape: (78, 64, 250)
Right-Resp Visual Cortex shape: (78, 135, 250)
time: 2.35 ms (started: 2023-07-19 23:31:22 +00:00)


In [28]:
Xs.keys()

dict_keys(['vis ctx', 'thal', 'hipp', 'other ctx'])

time: 4.36 ms (started: 2023-07-19 23:53:16 +00:00)
